#### Project Overview

This is the first Captone project for SpringBoard using the UCI dataset: Diabetes 130-US hospitals for years 1999-2008 Data Set. Detailed description of the dataset can be found here - https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008.

### Step One - Data Wrangling
First step is to read in the CSV file and explore the data to check for:
1) non-numerical values
2) missing cells

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [168]:
df=pd.read_csv('/Users/YingShen/Desktop/Data_Science/SpringBorad/Git/dataset_diabetes/diabetic_data.csv')

In [142]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

#### Feature selection
Before using Pandas to do data wrangling, EXCEL itself can be useful to filter out some features that have too many missingness, including:

weight: 98569;
payer_code: 40256;
medical_specialty: 49949

In [177]:
df=df[['patient_nbr', 'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'change', 'diabetesMed', 'readmitted']]
df.drop_duplicates(subset=["patient_nbr","gender"],inplace=True)
df["gender_d"]=df["gender"].apply(lambda x: 1 if x=="Male" else 0)
age_dict={"[0-10)":1,"[10-20)":2,"[20-30)":3,"[30-40)":4,"[40-50)":5,"[50-60)":6,
          "[60-70)":7,"[70-80)":8,"[80-90)":9,"[90-100)":10}
df["age_mapped"]=df["age"].map(age_dict)

In [178]:
df=df[df['discharge_disposition_id'].isin([11,13,14,19,20,21])==False]
df['dischar_home']=df['discharge_disposition_id'].apply(lambda x: 1 if x==1 or x==6 or x==8 else 0)

In [179]:
df['admission_source_id'].value_counts()
def ad_cource_group(value):
    if value==7 or value==25:
        return 1
    if 1<=value<=6 or value==10 or value==18 or value==22 or value==26:
        return 2
    else: return 3
df['admis_source_group']=df['admission_source_id'].apply(ad_cource_group)

In [180]:
df['max_glu_dummy']=df['max_glu_serum'].apply(lambda x: 0 if x=="None" else 1)
df['A1Cresult_dummy']=df['A1Cresult'].apply(lambda x: 0 if x=="None" else 1)
df['change_d']=df['change'].apply(lambda x: 1 if x=="Ch" else 0)
df['diabetesMed_d']=df['diabetesMed'].apply(lambda x: 1 if x=="Yes" else 0)
df['readmitted_d']=df['readmitted'].apply(lambda x: 1 if x=="Yes" else 0)

In [181]:
cleaned_df=df[['race', 'gender_d', 'age_mapped', 'dischar_home','admis_source_group',
               'time_in_hospital', 'num_lab_procedures', 'num_procedures', 
               'num_medications','number_outpatient', 'number_emergency', 
               'number_inpatient', 'diag_1','diag_2', 'diag_3', 'number_diagnoses', 
               'max_glu_dummy', 'A1Cresult_dummy','change_d', 'diabetesMed_d', 'readmitted_d']]

In [182]:
cleaned_df

race  gender_d  age_mapped  dischar_home  \
0             Caucasian         0           1             0   
1             Caucasian         0           2             1   
2       AfricanAmerican         0           3             1   
3             Caucasian         1           4             1   
4             Caucasian         1           5             1   
5             Caucasian         1           6             1   
6             Caucasian         1           7             1   
7             Caucasian         1           8             1   
8             Caucasian         0           9             1   
9             Caucasian         0          10             0   
10      AfricanAmerican         0           5             1   
11      AfricanAmerican         1           7             1   
12            Caucasian         0           5             0   
13            Caucasian         1           9             1   
14      AfricanAmerican         0           7             1   
15      AfricanAmerican         1           7             0   
16      AfricanAmerican         1           6             1   
17            Caucasian         0           6             1   
18      AfricanAmerican         1           8             1   
19                    ?         1           8             1   
20                    ?         0           6             1   
21                    ?         1           7             1   
22      AfricanAmerican         0           8             1   
23            Caucasian         0           9             1   
24      AfricanAmerican         0           8             1   
25                Other         0           6             1   
26            Caucasian         1           9             0   
27            Caucasian         0           6             1   
28            Caucasian         1           3             1   
29            Caucasian         0           9             0   
...                 ...       ...         ...           ...   
101715                ?         1           6             1   
101716            Other         1           8             0   
101717        Caucasian         1           6             0   
101718        Caucasian         0           8             1   
101719        Caucasian         1           2             0   
101720        Caucasian         1           6             1   
101721        Caucasian         0           8             0   
101722        Caucasian         0           8             1   
101723        Caucasian         1           7             1   
101724  AfricanAmerican         1           7             1   
101725        Caucasian         0           8             1   
101726         Hispanic         1           4             0   
101729        Caucasian         0           8             1   
101730  AfricanAmerican         0           8             1   
101731        Caucasian         1           6             1   
101732                ?         0           8             1   
101733        Caucasian         0           9             1   
101737        Caucasian         0           5             1   
101740        Caucasian         1           5             1   
101745        Caucasian         1           6             1   
101748        Caucasian         0           5             0   
101749        Caucasian         0           8             1   
101751        Caucasian         1           8             1   
101752            Other         0           5             1   
101753            Other         1           5             1   
101754        Caucasian         0           8             1   
101755            Other         0           5             1   
101756            Other         0           7             1   
101758        Caucasian         0           9             1   
101765        Caucasian         1           8             1   

        admis_source_group  time_in_hospital  num_lab_procedures  \
0                        2                 1